In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import collections
import math


#hyper parameters
batch_size = 100
learning_rate = 0.001
num_class = 10
num_epoch = 10

# cifar10 dataset
train_dataset = dsets.CIFAR10(root='./data', 
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = dsets.CIFAR10(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        #input channel, output channel, kernel_size
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv7 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(512)
        self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
    
    def forward(self, image):
        
        x = F.relu(self.conv1(image))  # 32x32x64
        x = self.bn1(x)
        x = F.relu(self.conv2(x))      # 32x32x64
        x = self.bn1(x)
        x = F.max_pool2d(x, 2)         # 16x16x64
        
        x = F.relu(self.conv3(x))  # 16x16x128
        x = self.bn2(x)
        x = F.relu(self.conv4(x))  # 16x16x128
        x = self.bn2(x)        
        x = F.max_pool2d(x, 2)     # 8x8x128
        
        x = F.relu(self.conv5(x))  # 8x8x256
        x = self.bn3(x)                
        x = F.relu(self.conv6(x))  # 8x8x256
        x = self.bn3(x)        
        x = F.relu(self.conv6(x))  # 8x8x256
        x = self.bn3(x)        
        x = F.max_pool2d(x, 2)     # 4x4x256
        
        x = F.relu(self.conv7(x))  # 4x4x512
        x = self.bn4(x)        
        x = F.relu(self.conv8(x))  # 4x4x512
        x = self.bn4(x)        
        x = F.relu(self.conv8(x))  # 4x4x512
        x = self.bn4(x)        
        x = F.max_pool2d(x, 2)     # 2x2x512
        
        x = F.relu(self.conv8(x))  # 2x2x512
        x = self.bn4(x)        
        x = F.relu(self.conv8(x))  # 2x2x512
        x = self.bn4(x)        
        x = F.relu(self.conv8(x))  # 2x2x512
        x = self.bn4(x)        
        x = F.max_pool2d(x, 2)     # 1x1x512
        
        x = x.view(-1, 512 * 1 * 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = self.fc3(x)

        return out
    


In [20]:
MODEL = VGG16()
print (MODEL)
MODEL.cuda(0)
criterion = nn.CrossEntropyLoss()
optimization = optim.Adam(MODEL.parameters(), lr=learning_rate)

VGG16(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
  (bn4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512,

In [21]:
# training step
for epoch in range(num_epoch):
    for i, (image, label) in enumerate(train_loader):
        images = Variable(image.cuda())
        labels = Variable(label.cuda())
        
        optimization.zero_grad()
        output = MODEL(images)
        loss = criterion(output, labels)
        loss.backward()
        optimization.step()
        
        if(i+1) % 100 == 0:
            print('epoch: [%d/%d], step: [%d/%d], Loss: %.4f' % (epoch+1, num_epoch, i+1, len(train_dataset)//batch_size, loss.data[0]))

epoch: [1/10], step: [100/500], Loss: 1.5726
epoch: [1/10], step: [200/500], Loss: 1.3986
epoch: [1/10], step: [300/500], Loss: 1.3671
epoch: [1/10], step: [400/500], Loss: 1.3389
epoch: [1/10], step: [500/500], Loss: 1.1497
epoch: [2/10], step: [100/500], Loss: 1.1523
epoch: [2/10], step: [200/500], Loss: 1.0123
epoch: [2/10], step: [300/500], Loss: 1.0870
epoch: [2/10], step: [400/500], Loss: 0.7743
epoch: [2/10], step: [500/500], Loss: 0.6848
epoch: [3/10], step: [100/500], Loss: 0.6387
epoch: [3/10], step: [200/500], Loss: 0.8468
epoch: [3/10], step: [300/500], Loss: 0.6351
epoch: [3/10], step: [400/500], Loss: 0.7305
epoch: [3/10], step: [500/500], Loss: 0.8061
epoch: [4/10], step: [100/500], Loss: 0.6431
epoch: [4/10], step: [200/500], Loss: 0.4914
epoch: [4/10], step: [300/500], Loss: 0.6404
epoch: [4/10], step: [400/500], Loss: 0.6541
epoch: [4/10], step: [500/500], Loss: 0.5077
epoch: [5/10], step: [100/500], Loss: 0.5216
epoch: [5/10], step: [200/500], Loss: 0.4831
epoch: [5/

In [22]:
total = 0
correct = 0

for (image, label) in test_loader:
    images = Variable(image.cuda())
    
    output = MODEL(images)
    _, predicted = torch.max(output.data, 1)
    correct += (predicted.cpu()==label).sum()
    total += labels.size(0)

print('Accuracy : %d%%' %((100*correct)/total))

Accuracy : 96%
